### Récupération des bibliothèques essentielles

In [2]:
import os
import requests
import json
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from PIL.ExifTags import TAGS
from io import BytesIO

### Pull de 100 oeuvres d'art (formats png, jpeg, etc.) depuis Wikidata

In [8]:
def create_images_folder():
    if not os.path.exists("images"):
        os.makedirs("images")
        print("Dossier 'images' créé.")

def get_wikidata_images(query):
    endpoint_url = "https://query.wikidata.org/sparql"
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def download_images_from_wikidata(num_images=100):
    query = """
    SELECT ?item ?itemLabel ?pic WHERE {
  ?item wdt:P31 wd:Q3305213;
    wdt:P18 ?pic.
}
LIMIT 110
    """
    
    results = get_wikidata_images(query)
    image_urls = [result["pic"]["value"] for result in results["results"]["bindings"]]
    downloaded_images = []
    
    # Définir les headers avec un User-Agent de type Mozilla
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    for idx, url in enumerate(image_urls[:num_images]):
        try:
            img_response = requests.get(url, headers=headers)  # Ajout des headers ici
            img = Image.open(BytesIO(img_response.content))
            
            extension = img.format.lower()
            filename = f"images/image_{idx}.{extension}"
            img.save(filename)
            downloaded_images.append(filename)
            print(f"Image enregistrée : {filename}")
        except Exception as e:
            print(f"Erreur lors du téléchargement de {url}: {e}")
    
    return downloaded_images

def extract_metadata(image_path):
    try:
        img = Image.open(image_path)
        metadata = {
            "filename": os.path.basename(image_path),
            "format": img.format,
            "size": img.size,
            "orientation": "Portrait" if img.size[1] > img.size[0] else "Paysage" if img.size[0] > img.size[1] else "Carré",
        }
        
        exif_data = img._getexif()
        if exif_data:
            for tag, value in exif_data.items():
                decoded = TAGS.get(tag, tag)
                metadata[decoded] = value
        
        return metadata
    except Exception as e:
        print(f"Erreur lors de l'extraction des métadonnées pour {image_path}: {e}")
        return {}

def save_metadata(image_files, output_json="metadata.json"):
    metadata_list = [extract_metadata(img) for img in image_files]
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(metadata_list, f, indent=4, ensure_ascii=False)
    print(f"Métadonnées enregistrées dans {output_json}")

def main():
    create_images_folder()
    images = download_images_from_wikidata()
    if images:
        save_metadata(images)
    else:
        print("Aucune image téléchargée.")

if __name__ == "__main__":
    main()


Dossier 'images' créé.
Image enregistrée : images/image_0.jpeg
Image enregistrée : images/image_1.jpeg
Image enregistrée : images/image_2.jpeg
Image enregistrée : images/image_3.jpeg
Image enregistrée : images/image_4.jpeg
Image enregistrée : images/image_5.jpeg
Erreur lors du téléchargement de http://commons.wikimedia.org/wiki/Special:FilePath/Portrait%20d%27une%20Femme%20%C3%A0%20sa%20Toilette%2C%20by%20Titian%2C%20from%20C2RMF%20retouched.jpg: Image size (229533750 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
Image enregistrée : images/image_7.jpeg
Image enregistrée : images/image_8.jpeg
Image enregistrée : images/image_9.jpeg
Image enregistrée : images/image_10.jpeg
Image enregistrée : images/image_11.jpeg
Image enregistrée : images/image_12.jpeg
Image enregistrée : images/image_13.jpeg
Image enregistrée : images/image_14.jpeg
Image enregistrée : images/image_15.jpeg
Image enregistrée : images/image_16.jpeg
Image enregistrée : images/image_17.j

/home/dev/.local/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (117833694 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Image enregistrée : images/image_58.jpeg
Image enregistrée : images/image_59.jpeg
Image enregistrée : images/image_60.jpeg
Image enregistrée : images/image_61.jpeg
Image enregistrée : images/image_62.jpeg
Image enregistrée : images/image_63.jpeg
Image enregistrée : images/image_64.jpeg
Image enregistrée : images/image_65.jpeg
Image enregistrée : images/image_66.jpeg
Image enregistrée : images/image_67.jpeg
Image enregistrée : images/image_68.jpeg
Image enregistrée : images/image_69.jpeg
Image enregistrée : images/image_70.jpeg


/home/dev/.local/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (178746750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Image enregistrée : images/image_71.jpeg
Image enregistrée : images/image_72.jpeg
Image enregistrée : images/image_73.jpeg
Image enregistrée : images/image_74.jpeg
Image enregistrée : images/image_75.jpeg
Erreur lors du téléchargement de http://commons.wikimedia.org/wiki/Special:FilePath/Las%20Meninas%2C%20by%20Diego%20Vel%C3%A1zquez%2C%20from%20Prado%20in%20Google%20Earth.jpg: Image size (781950000 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
Image enregistrée : images/image_77.jpeg
Image enregistrée : images/image_78.mpo
Image enregistrée : images/image_79.jpeg
Image enregistrée : images/image_80.jpeg
Image enregistrée : images/image_81.jpeg
Erreur lors du téléchargement de http://commons.wikimedia.org/wiki/Special:FilePath/The%20Night%20Watch%20-%20HD.jpg: Image size (2800346094 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.
Image enregistrée : images/image_83.jpeg
Image enregistrée : images/image_84.jpeg
Imag

### Récupération des couleurs prédominantes sur chaque image

In [ ]:
import os
import json
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def get_dominant_color(image_path, n_colors=3):
    # Ouvrir l'image et s'assurer qu'elle est bien en RGB
    if Image.open(image_path):
        img = Image.open(image_path).convert('RGB')
    
    # Redimensionner l'image pour accélérer le traitement
    img = img.resize((img.width // 10, img.height // 10))  # Ajuste la taille pour plus de rapidité
    
    # Convertir l'image en un tableau numpy
    img_array = np.array(img)
    
    # Reshaper l'image pour une liste de pixels (chaque pixel devient une ligne dans le tableau)
    pixels = img_array.reshape(-1, 3)
    
    # Appliquer KMeans pour trouver les couleurs dominantes
    kmeans = KMeans(n_clusters=n_colors)
    kmeans.fit(pixels)
    
    # Récupérer les couleurs dominantes (les centres des clusters)
    dominant_colors = kmeans.cluster_centers_.astype(int)
    
    # Convertir les couleurs en format hexadécimal
    dominant_colors_hex = [mcolors.rgb2hex(c / 255.0) for c in dominant_colors]
    
    return dominant_colors_hex

def process_images_in_folder(folder_path, n_colors=3):
    dominant_colors_per_image = {}
    
    # Liste des fichiers dans le dossier
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Vérifier si c'est une image (tu peux ajouter plus de formats ici si nécessaire)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                dominant_colors = get_dominant_color(file_path, n_colors)
                dominant_colors_per_image[filename] = dominant_colors
                print(f"{filename}: {dominant_colors}")
            except Exception as e:
                print(f"Erreur lors du traitement de {filename}: {e}")
    
    return dominant_colors_per_image

def update_metadata_with_colors(metadata_path, folder_path, n_colors=3):
    # Charger les métadonnées existantes
    try:
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)
    except Exception as e:
        print(f"Erreur lors du chargement de {metadata_path}: {e}")
        return

    # Extraire les couleurs dominantes pour chaque image dans le dossier
    dominant_colors_per_image = process_images_in_folder(folder_path, n_colors)
    
    # Mettre à jour les métadonnées avec la clé 'colors'
    for entry in metadata:
        if 'filename' in entry:
            filename = entry['filename']
            if filename in dominant_colors_per_image:
                entry['colors'] = dominant_colors_per_image[filename]
        else:
            print(f"Entrée invalide sans 'filename' : {entry}")

    
    # Sauvegarder les métadonnées mises à jour dans le fichier
    try:
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=4, ensure_ascii=False)
        print(f"Les métadonnées ont été mises à jour avec les couleurs.")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde des métadonnées : {e}")

def main():
    metadata_path = "metadata.json"  # Chemin vers le fichier metadata
    folder_path = "images"  # Dossier contenant les images
    update_metadata_with_colors(metadata_path, folder_path)
    
if __name__ == "__main__":
    main()


image_25.jpeg: ['#fcfbfa', '#b1885f', '#503d38']
image_16.jpeg: ['#c1afa7', '#362322', '#865a46']
image_70.jpeg: ['#7e765d', '#3a3326', '#bec7ba']
image_2.jpeg: ['#81694d', '#37302b', '#bfa77d']
image_26.jpeg: ['#181a1f', '#b99e80', '#6a4634']


/home/dev/.local/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (117833694 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


image_58.jpeg: ['#c7aa98', '#201a16', '#84594a']
image_20.jpeg: ['#7a7579', '#352d29', '#c0c3c7']
image_97.jpeg: ['#907861', '#41332c', '#cdbeb2']
image_41.jpeg: ['#312b2b', '#bc8261', '#6e453f']
image_47.jpeg: ['#5f3f27', '#756e5b', '#2f2921']
image_32.jpeg: ['#503c39', '#c7bca8', '#806d66']
image_17.jpeg: ['#4a4c37', '#b0bdcc', '#707e8e']
image_83.jpeg: ['#252315', '#4b3425', '#817058']
image_89.jpeg: ['#352e16', '#635436', '#9c8566']
image_51.jpeg: ['#1f221e', '#a29373', '#564f3c']
image_53.jpeg: ['#151213', '#e1e0cf', '#90856d']
image_63.jpeg: ['#c7cac5', '#2c2d23', '#798076']
image_29.jpeg: ['#734841', '#2a1116', '#9c7c61']
image_22.jpeg: ['#354049', '#181a20', '#8d8082']
image_5.jpeg: ['#bfb7a5', '#3e2b24', '#9c542c']
image_87.jpeg: ['#948c7d', '#382a26', '#4d545b']
image_72.jpeg: ['#74521d', '#2f1006', '#b9ac8f']
image_27.jpeg: ['#30291c', '#929186', '#5d5c4f']
image_19.jpeg: ['#2b1a1b', '#56392d', '#cfaf7f']
image_30.jpeg: ['#574033', '#2b1a17', '#907352']
image_36.jpeg: ['#dbd

/home/dev/.local/lib/python3.10/site-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (178746750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


image_71.jpeg: ['#655c49', '#261f14', '#bdaa8f']
image_55.jpeg: ['#2b2a1d', '#5e5c3f', '#a8a080']
image_39.jpeg: ['#36322d', '#7a7063', '#beb5a9']
image_4.jpeg: ['#33271c', '#765b33', '#b99357']
image_59.jpeg: ['#32302a', '#928763', '#595340']
image_79.jpeg: ['#584427', '#af8e5a', '#211810']
image_91.jpeg: ['#23140b', '#b1a380', '#6f4821']
image_49.jpeg: ['#ded3b1', '#b35c30', '#492819']
image_99.jpeg: ['#9b8e7a', '#362823', '#634a42']
image_31.jpeg: ['#34311f', '#909e95', '#696548']
image_14.jpeg: ['#352f25', '#a3886c', '#5f5250']
image_65.jpeg: ['#2f2924', '#cfc6b1', '#89755c']
image_21.jpeg: ['#f3eee8', '#a68c72', '#55372a']
image_98.jpeg: ['#1c0d07', '#c2ab8e', '#6e4526']
image_85.jpeg: ['#28231a', '#c3a575', '#6a5b44']
image_61.jpeg: ['#a5b1c3', '#203e48', '#5d767f']
image_13.jpeg: ['#342f24', '#baac93', '#837963']
image_18.jpeg: ['#706d5d', '#3e1410', '#948f79']
image_9.jpeg: ['#c0bfb1', '#33281b', '#776440']
image_38.jpeg: ['#312e29', '#a79883', '#5b5343']
image_69.jpeg: ['#2318

KeyError: 'filename'

Test API github JSAMUEL

In [ ]:
import requests
url="https://api.github.com/users/johnsamuelwrites"

response = requests.get(url)
print(response.json())

[{'login': 'mojombo', 'id': 1, 'node_id': 'MDQ6VXNlcjE=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/mojombo', 'html_url': 'https://github.com/mojombo', 'followers_url': 'https://api.github.com/users/mojombo/followers', 'following_url': 'https://api.github.com/users/mojombo/following{/other_user}', 'gists_url': 'https://api.github.com/users/mojombo/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/mojombo/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/mojombo/subscriptions', 'organizations_url': 'https://api.github.com/users/mojombo/orgs', 'repos_url': 'https://api.github.com/users/mojombo/repos', 'events_url': 'https://api.github.com/users/mojombo/events{/privacy}', 'received_events_url': 'https://api.github.com/users/mojombo/received_events', 'type': 'User', 'user_view_type': 'public', 'site_admin': False}, {'login': 'defunkt', 'id': 2, 'node_id': 'MDQ6VXNlcjI=', 'ava